# Data Collection from API

In [1]:
!pip install requests


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import requests
import pandas as pd
from ta import add_all_ta_features
import matplotlib.pyplot as plt

In [2]:
def process_json(json_data):
    # Extracting the 'data' part of the JSON and creating a DataFrame
    data_list = json_data['data']
    processed_data = []

    # Loop through each entry in 'data' to flatten the structure
    """Retrieved from: 
    https://www.programiz.com/python-programming/methods/dictionary/pop
    https://www.w3schools.com/python/ref_dictionary_update.asp"""
    for entry in data_list:
        entry_data = entry.pop('data')  # Remove the 'data' key
        entry.update(entry_data)  # Update the entry with the 'data' content
        processed_data.append(entry)
        
    return processed_data


def get_data_from_api(api_endpoint):
    # Make a GET request to the API endpoint
    """Retrieved from: https://www.codingthesmartway.com/how-to-get-data-from-rest-api-with-python/"""
    response = requests.get(api_endpoint)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Access the data from the response
        json_data = response.json()  # Convert response to JSON format
        # Now 'data' contains the fetched information
#         print(json_data)
        return json_data
    else:
        print("Failed to fetch data:", response.status_code)
        
def transform_datetime(df):
    # Convert the timestamp column to datetime format
    """Retrieved from: 
    https://www.geeksforgeeks.org/convert-the-column-type-from-string-to-datetime-format-in-pandas-dataframe/
    """
    df['date'] = pd.to_datetime(df['date'])

    # Extract year, month, day, and hour into separate columns
    """Retrieved from:     
    https://stackoverflow.com/questions/25146121/extracting-just-month-and-year-separately-from-pandas-datetime-column
    """
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['hour'] = df['date'].dt.hour
    
def get_last_date(df):
    last_year_value = df.iloc[-1]['year']
    last_month_value = df.iloc[-1]['month']
    last_day_value = df.iloc[-1]['day']
    
    # Convert month and day values to strings with leading zeros if < 10 so there's no confilcts later
    """Retrieved from: https://ioflood.com/blog/python-zfill/#:~:text=The%20zfill()%20method%20in,the%20actual%20number%20they%20represent."""
    last_month_str = str(last_month_value).zfill(2)  # Add leading zero if necessary
    last_day_str = str(last_day_value).zfill(2)  # Add leading zero if necessary
    
    last_date = str(last_year_value) + '-' + last_month_str + '-' + last_day_str
        
    return last_date

# Get data by 180 day batches from 2018-10-10 up to 2023-12-01

In [3]:
def get_whole_df(company_ticker, api_token):
    last_date = '2023-12-01'

    for i in range(25):
        # URL of the API endpoint you want to fetch data from
        api_endpoint = f"https://api.stockdata.org/v1/data/intraday?symbols={company_ticker}&interval=hour&date_to={last_date}&api_token={api_token}"

        json_data = get_data_from_api(api_endpoint)

        # Process idented data
        processed_data = process_json(json_data)

        if(i == 0):
            # Convert the processed data into a DataFrame
            df = pd.DataFrame(processed_data)

            transform_datetime(df)

            # Get last day based on iteration
            last_date = get_last_date(df)

        else: 
            # Convert the data on this iteration into a df
            df_temp = pd.DataFrame(processed_data)

            transform_datetime(df_temp)

            # Get last day based on iteration
            last_date = get_last_date(df_temp)

            # Concatenate this data get with the whole df
            df = pd.concat([df, df_temp], ignore_index=True)
            
    return df

In [ ]:
# Change for your API token
api_token = "JrBSlK0H1s7uGHhTyl56kxjYSGfSikPl5YJmvIv0"

# Apple stocks data

In [4]:
aapl_df = get_whole_df('AAPL', api_token)

In [5]:
aapl_df

,date,ticker,open,high,low,close,volume,is_extended_hours,year,month,day,hour
0,2023-12-01 16:00:00+00:00,AAPL,191.21,191.21,191.20,191.21,202,False,2023,12,1,16
1,2023-12-01 15:00:00+00:00,AAPL,191.28,191.38,190.88,191.15,182247,False,2023,12,1,15
2,2023-12-01 14:00:00+00:00,AAPL,191.21,191.34,190.92,191.27,100877,False,2023,12,1,14
3,2023-12-01 13:00:00+00:00,AAPL,191.52,191.55,190.83,191.17,122182,False,2023,12,1,13
4,2023-12-01 12:00:00+00:00,AAPL,191.05,191.54,190.99,191.52,118471,False,2023,12,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...
9789,2018-10-10 13:00:00+00:00,AAPL,222.51,223.80,222.18,222.75,88034,False,2018,10,10,13
9790,2018-10-10 12:00:00+00:00,AAPL,223.37,223.94,222.56,222.57,71484,False,2018,10,10,12
9791,2018-10-10 11:00:00+00:00,AAPL,223.53,224.39,222.19,223.41,153183,False,2018,10,10,11
9792,2018-10-10 10:00:00+00:00,AAPL,223.55,224.34,222.59,223.56,158247,False,2018,10,10,10


In [6]:
# Change for your local file path
file_path = r"C:\Users\berna\Documents\ML Project-CS50 Final\Finance-CompanyStocks\API Gathered Data\apple_stocks.csv"

# Export the DataFrame to a CSV file with the specified path
"""Retrieved from: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html"""
aapl_df.to_csv(file_path, index=False)

# Tesla stocks data

In [7]:
tsla_df = get_whole_df('TSLA', api_token)

In [8]:
tsla_df

,date,ticker,open,high,low,close,volume,is_extended_hours,year,month,day,hour
0,2023-12-01 16:00:00+00:00,TSLA,238.75,238.75,238.75,238.75,400,False,2023,12,1,16
1,2023-12-01 15:00:00+00:00,TSLA,238.62,239.07,237.64,238.85,76279,False,2023,12,1,15
2,2023-12-01 14:00:00+00:00,TSLA,237.30,238.76,236.69,238.64,38070,False,2023,12,1,14
3,2023-12-01 13:00:00+00:00,TSLA,238.78,239.72,237.31,237.39,27618,False,2023,12,1,13
4,2023-12-01 12:00:00+00:00,TSLA,237.33,240.16,237.28,238.72,41528,False,2023,12,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...
9587,2018-10-10 13:00:00+00:00,TSLA,253.99,261.99,253.31,258.85,45747,False,2018,10,10,13
9588,2018-10-10 12:00:00+00:00,TSLA,255.67,258.36,253.95,254.60,23954,False,2018,10,10,12
9589,2018-10-10 11:00:00+00:00,TSLA,251.41,258.73,248.99,255.17,55080,False,2018,10,10,11
9590,2018-10-10 10:00:00+00:00,TSLA,248.06,254.40,247.81,251.78,50254,False,2018,10,10,10


In [9]:
# Change for your local file path
file_path = r"C:\Users\berna\Documents\ML Project-CS50 Final\Finance-CompanyStocks\API Gathered Data\tesla_stocks.csv"

# Export the DataFrame to a CSV file with the specified path
tsla_df.to_csv(file_path, index=False)

# Microsoft stocks data

In [10]:
msft_df = get_whole_df('MSFT', api_token)

In [11]:
msft_df

,date,ticker,open,high,low,close,volume,is_extended_hours,year,month,day,hour
0,2023-12-01 16:00:00+00:00,MSFT,374.30,374.30,374.30,374.30,200,False,2023,12,1,16
1,2023-12-01 15:00:00+00:00,MSFT,375.20,375.50,373.72,374.28,148512,False,2023,12,1,15
2,2023-12-01 14:00:00+00:00,MSFT,373.09,375.28,372.83,375.19,61819,False,2023,12,1,14
3,2023-12-01 13:00:00+00:00,MSFT,374.73,374.81,372.92,373.10,107670,False,2023,12,1,13
4,2023-12-01 12:00:00+00:00,MSFT,373.14,375.29,372.25,374.69,149653,False,2023,12,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...
9599,2018-10-10 13:00:00+00:00,MSFT,108.50,109.41,108.12,108.68,157180,False,2018,10,10,13
9600,2018-10-10 12:00:00+00:00,MSFT,108.19,109.24,107.85,108.53,78664,False,2018,10,10,12
9601,2018-10-10 11:00:00+00:00,MSFT,109.23,109.42,107.68,108.23,181129,False,2018,10,10,11
9602,2018-10-10 10:00:00+00:00,MSFT,109.57,109.84,108.94,109.31,179593,False,2018,10,10,10


In [12]:
# Change for your local file path
file_path = r"C:\Users\berna\Documents\ML Project-CS50 Final\Finance-CompanyStocks\API Gathered Data\microsoft_stocks.csv"

# Export the DataFrame to a CSV file with the specified path
msft_df.to_csv(file_path, index=False)

# Google stocks data

In [13]:
googl_df = get_whole_df('GOOGL', api_token)

In [14]:
googl_df

,date,ticker,open,high,low,close,volume,is_extended_hours,year,month,day,hour
0,2023-12-01 16:00:00+00:00,GOOGL,131.82,131.82,131.82,131.82,171,False,2023,12,1,16
1,2023-12-01 15:00:00+00:00,GOOGL,131.67,131.96,131.50,131.82,262688,False,2023,12,1,15
2,2023-12-01 14:00:00+00:00,GOOGL,131.46,131.74,131.30,131.68,114173,False,2023,12,1,14
3,2023-12-01 13:00:00+00:00,GOOGL,131.88,131.95,131.38,131.47,127057,False,2023,12,1,13
4,2023-12-01 12:00:00+00:00,GOOGL,131.40,132.01,131.32,131.85,160325,False,2023,12,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...
9557,2018-10-10 13:00:00+00:00,GOOGL,1117.77,1128.94,1116.70,1121.30,7628,False,2018,10,10,13
9558,2018-10-10 12:00:00+00:00,GOOGL,1118.36,1124.51,1115.00,1120.55,12958,False,2018,10,10,12
9559,2018-10-10 11:00:00+00:00,GOOGL,1121.77,1122.15,1113.58,1118.21,14780,False,2018,10,10,11
9560,2018-10-10 10:00:00+00:00,GOOGL,1119.94,1125.47,1117.19,1121.45,18739,False,2018,10,10,10


In [15]:
# Change for your local file path
file_path = r"C:\Users\berna\Documents\ML Project-CS50 Final\Finance-CompanyStocks\API Gathered Data\google_stocks.csv"

# Export the DataFrame to a CSV file with the specified path
googl_df.to_csv(file_path, index=False)